# Training ANN for predicting shear resistance of concrete beams reinforced by FRP bars

Reference: Shear resistance prediction of concrete beams reinforced by FRP bars using artificial neural networks
https://doi.org/10.1016/j.measurement.2018.05.051

The data from Table A1 of the above paper are saved into train_data.xlsx and loaded in this script for training an ANN using two different training algortihms: Adam and Levenberg-Marquardt. The data contains 6 input variables and one output variable. The latter is the shear resistance calculated according to ACI440.1R.

The fitting is done with a shallow neural network. This script is adjusted from the script published at https://github.com/fabiodimarco/tf-levenberg-marquardt . In this script it is shown that first-order methods such as Adam fail to converge, while Levenberg–Marquardt converges rapidly with very low loss values. The values of learning_rate were chosen experimentally on the basis of the results obtained by each algorithm.


In [21]:
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
# If the module levenberg_marquardt cannot be imported in Google Collab, then You can do this by clicking on the “Files” button on the left-hand side of the Colab interface, and then clicking on the “Upload” button. Once you have uploaded the module file (levenberg_marquardt.py), you can import it in your Colab notebook using a standard Python import statement.
import levenberg_marquardt as lm

In [22]:
input_size = 177  # Update this to match the size of your dataset
batch_size = 18
# Get the directory where this script is located
script_dir=os.getcwd()
print(script_dir)
# Construct the relative path to the Excel file
relative_file_path = 'train_data.xlsx'
excel_file_path = os.path.join(script_dir, relative_file_path)
print(excel_file_path)
# Load training data from Excel file
# If the data cannot be loaded in Google Collab, then upload the Excel file by clicking on the “Files” button on the left-hand side of the Colab interface, and then clicking on the “Upload” button.
train_data = pd.read_excel(excel_file_path, skiprows=0)
# Print dimensions of the data being loaded
print(train_data.shape)
# Separate input and output training data
x_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
# Print dimensions of the x data
print(x_train.shape)
# Print dimensions of the y data
print(y_train.shape)
#print(x_train)
#print(y_train)


c:\Users\gpapa\OneDrive\Desktop\10_GITHUB_REPOSITORIES_AND_CODE\tf-levenberg-marquardt-main
c:\Users\gpapa\OneDrive\Desktop\10_GITHUB_REPOSITORIES_AND_CODE\tf-levenberg-marquardt-main\train_data.xlsx
(177, 7)
(177, 6)
(177,)


In [23]:
# Convert data to TensorFlow tensors
x_train = tf.constant(x_train, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
print(x_train)
print(y_train)

tf.Tensor(
[[200.   325.     3.2   44.6    0.7  137.  ]
 [200.   325.     3.2   45.     0.7  137.  ]
 [200.   325.     3.2   46.9    0.9   58.  ]
 ...
 [150.   215.5    3.    40.3    0.44  41.  ]
 [150.   213.5    3.    40.3    0.44  40.  ]
 [150.   213.5    3.    40.3    0.44  40.  ]], shape=(177, 6), dtype=float32)
tf.Tensor(
[157.5312   158.15695  148.00992  143.87836  119.98278   71.727554
  82.13615   84.35938   87.51099   86.419235 118.12378   58.613304
  62.451855 119.02614   93.1749    80.39414  102.76702  102.76702
 102.76702   83.034065  83.034065  83.034065 109.17473  109.17473
 109.17473  134.50493  134.50493  134.50493  123.74944  123.74944
 123.74944  112.84444  112.84444  112.84444   90.908844  80.59229
  88.00093  332.09488  328.11786  343.30484  379.92804  377.059
 103.73373  103.73373  119.33115  119.33115  106.75536  106.75536
  89.169334  89.169334  99.386696  99.386696  91.321236  91.321236
 128.3658   128.3658   128.3658   101.17766  101.17766  101.17766
 113.4502

In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(input_size)
train_dataset = train_dataset.batch(batch_size).cache()
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
print(train_dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20, activation='tanh', input_shape=(6,)),
    tf.keras.layers.Dense(1, activation='linear')])

model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss=tf.keras.losses.MeanSquaredError())

model_wrapper = lm.ModelWrapper(
    tf.keras.models.clone_model(model))

model_wrapper.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1.0),
    loss=lm.MeanSquaredError())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 20)                140       
                                                                 
 dense_5 (Dense)             (None, 1)                 21        
                                                                 
Total params: 161 (644.00 Byte)
Trainable params: 161 (644.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
print("Train using Adam")
t1_start = time.perf_counter()
model.fit(train_dataset, epochs=1000)
t1_stop = time.perf_counter()
print("Elapsed time: ", t1_stop - t1_start)


Train using Adam
Epoch 1/1000


10/10 [==============================] - 0s 1ms/step - loss: 23743.8496
Epoch 2/1000
10/10 [==============================] - 0s 999us/step - loss: 23048.6348
Epoch 3/1000
10/10 [==============================] - 0s 962us/step - loss: 22445.8340
Epoch 4/1000
10/10 [==============================] - 0s 896us/step - loss: 21954.7012
Epoch 5/1000
10/10 [==============================] - 0s 889us/step - loss: 21484.9707
Epoch 6/1000
10/10 [==============================] - 0s 889us/step - loss: 21017.7109
Epoch 7/1000
10/10 [==============================] - 0s 1ms/step - loss: 20560.6406
Epoch 8/1000
10/10 [==============================] - 0s 982us/step - loss: 20117.0254
Epoch 9/1000
10/10 [==============================] - 0s 786us/step - loss: 19687.3145
Epoch 10/1000
10/10 [==============================] - 0s 890us/step - loss: 19271.1719
Epoch 11/1000
10/10 [==============================] - 0s 889us/step - loss: 18868.0957
Epoch 12/1000
10/10 [==============================] - 0s 

In [27]:
print("\n_________________________________________________________________")
print("Train using Levenberg-Marquardt")
t2_start = time.perf_counter()
model_wrapper.fit(train_dataset, epochs=100)
t2_stop = time.perf_counter()
print("Elapsed time: ", t2_stop - t2_start)



_________________________________________________________________
Train using Levenberg-Marquardt
Epoch 1/100
10/10 [==============================] - 0s 31ms/step - damping_factor: 1.0000e-07 - attempts: 1.0000 - loss: 6633.2704
Epoch 2/100
10/10 [==============================] - 0s 18ms/step - damping_factor: 1.0000e-07 - attempts: 1.0000 - loss: 6607.7976
Epoch 3/100
10/10 [==============================] - 0s 19ms/step - damping_factor: 1.0000e-07 - attempts: 2.0000 - loss: 6397.6850
Epoch 4/100
10/10 [==============================] - 0s 19ms/step - damping_factor: 1.0000e-07 - attempts: 2.0000 - loss: 6775.3548
Epoch 5/100
10/10 [==============================] - 0s 19ms/step - damping_factor: 1.0000e-07 - attempts: 2.0000 - loss: 6724.4582
Epoch 6/100
10/10 [==============================] - 0s 20ms/step - damping_factor: 1.0000e-07 - attempts: 2.0000 - loss: 6544.6706
Epoch 7/100
10/10 [==============================] - 0s 19ms/step - damping_factor: 1.0000e-07 - attempts: 2.

In [28]:
# print("\n_________________________________________________________________")
# print("Plot results")
# plt.plot(x_train, y_train, 'b-', label="reference")
# plt.plot(x_train, model.predict(x_train), 'g--', label="adam")
# plt.plot(x_train, model_wrapper.predict(x_train), 'r--', label="lm")
# plt.legend()
# plt.show()
